# Preparation

## Library installation/import

Install and import libraries that are used in multiple sections of the pipeline.

In [1]:
import pandas as pd

In [2]:
!pip install spacy==3.0
!pip install tweepy
!pip install wordcloud

     |████████████████████████████████| 12.4 MB 9.0 MB/s eta 0:00:011    |██████████████                  | 5.5 MB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 41.4 MB/s eta 0:00:01
  Using cached catalogue-2.0.1-py3-none-any.whl (9.6 kB)
     |████████████████████████████████| 106 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 36.5 MB/s eta 0:00:01
  Using cached spacy_legacy-3.0.1-py2.py3-none-any.whl (7.0 kB)
     |████████████████████████████████| 450 kB 3.2 MB/s eta 0:00:01     |█████████████                   | 184 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 2.3 MB/s eta 0:00:01
  Using cached typer-0.3.2-py3-none-any.whl (21 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached pathy-0.4.0-py3-none-any.whl (36 kB)
  Using cached smart_open-3.0.0.tar.gz (113 kB)
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=52bbb286e5239ead736470dfb1a3995e90

  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp36-cp36m-macosx_10_9_x86_64.whl size=161048 sha256=b862fe21a5ce6e775d638996a2d89edd668626d384656adb430aa29dd369b1d7
  Stored in directory: /Users/Radha/Library/Caches/pip/wheels/09/7a/57/49f0c5fb0d6aaee90cb8c2d13d09a909a7b0ce42f8805fe69a
Successfully built wordcloud


## Configuring Twitter API keys

Please note that the API keys below are the course leader's own API keys. You are allowed to use it to do some small tests, but please be careful because all students in the class now have a copy of it, and hence the limits can be easily exceeded.

If your group has decided to use Twitter data, you can [apply for your own keys](https://developer.twitter.com/en/apply-for-access).

In [ ]:
import tweepy

auth = tweepy.OAuthHandler('fjkruboMzTLE4BLE7FmEpkWpw', 'jDobYz45Ksc3uMHoD2QnyZK60NwfRZnWIDVmyPtUGLkiOUqfGl')
auth.set_access_token('1374773661861830658-lPZKU2qeuepRxVfWs5OxRoZd6XGzrH', '84k6xIDIMrt5mzPFLCoesD0WM9bpk8d3bAKaNonbcuT0s')

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Downloading of new data set

**IMPORTANT NOTE**: Please do not run the cell below unless intending to download a new data set.

Make sure that you change the parameters.

Also, check the [Tweepy API reference](https://docs.tweepy.org/en/latest/api.html) to find out about other ways through which you can retrieve tweets, e.g., by specifying usernames.

In [ ]:
# Collect tweets
query = "#notoracism" + " -filter:retweets"
cutoff_date = "2021-01-01"
tweets = tweepy.Cursor(api.search, q=query, lang="en", since=cutoff_date).items(1000)

tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]

tweets_df = pd.DataFrame(data=tweets_list, columns=['date', 'user', 'location', 'text'])

# A good idea to save downloaded tweets as CSV
tweets_df.to_csv ('current_set.csv', quotechar='"', encoding='utf8', index = False, header=True)

# Data pre-processing

## Cleaning

Below we provide some code for text cleaning. However, we encourage you to think of other ways to clean your data, e.g., by removing hashtags, removing usernames, removing duplicate tweets?

In [52]:
#installing tweet-preprocessor
!pip install tweet-preprocessor
!pip install wordsegment
import preprocessor as p
from wordsegment import load, segment
import re

     |████████████████████████████████| 4.8 MB 883 kB/s eta 0:00:01


In [74]:
# comment if not using pre saved data set 
tweets_df = pd.read_csv('current_set.csv', quotechar='"', encoding='utf8')
cleaned_tweets = []

# include hastags and remove urls and emojis and mentions
#p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
for tweet in tweets_df['text']:
    cleaned_tweets.append(p.clean(tweet))

# adding back to data frame
tweets_df['text_processed'] = cleaned_tweets

#removing punctuation
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub('[,\\.!?]', '', x))

# Remove unnecessary line breaks
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r"\n", '', x))

# Convert the titles to lowercase
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: x.lower())

#removing hastags for now 
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: re.sub(r"#(\w+)","",x))

#removing extra whitespaces 
tweets_df['text_processed'] = tweets_df['text_processed'].map(lambda x: ' '.join(x.split()))


display(tweets_df)

,date,user,location,text,text_processed
0,2021-03-06 10:35:34,RWDMolenbeek,"Sint-Jans-Molenbeek, België",🏳️‍🌈Matchday: R.W.D.M - @KMSKDeinze 🏟E. Machte...,matchday: rwdm - e machtens stadion 20:45
1,2021-03-06 10:24:42,motsetse_sello,NaN,I've never seen I a white Human ever since I w...,i've never seen i a white human ever since i w...
2,2021-03-06 07:24:48,amirjon4628,NaN,"Hi army,This is Iranian armys\nWe just found m...",hi armythis is iranian armys we just found meh...
3,2021-03-06 07:12:20,future_nostalgi,NaN,"Hi army,This is Iranian armys\nWe just found m...",hi armythis is iranian armys we just found meh...
4,2021-03-06 07:09:39,NJ7twt,NaN,@Nili10724948 @MehradHidden Whenever the hater...,whenever the hater starts to hate bts does not...
5,2021-03-06 07:06:38,Xhtly1,NaN,"@BunyaratwanitB @BTS_twt Hi army,This is Irani...",hi armythis is iranian armys we just found meh...
6,2021-03-06 06:57:56,_im_pudding_,NaN,RACISM IS NOT AN OPINION \n#Matuschik_OUT\n#N...,racism is not an opinion
7,2021-03-06 06:56:48,smartzyofficial,"Lagos, Nigeria",A Thread 👇 👇 👇 #NOtoRacism #AlperenDuymaz http...,a thread
8,2021-03-06 06:48:14,Mohi81933368,NaN,"Hi army, This is Iranian armys\nWe just found ...",hi army this is iranian armys we just found th...
9,2021-03-06 06:25:25,Xhtly1,NaN,"@jjk_mae Hi army,This is Iranian armys\nWe jus...",hi armythis is iranian armys we just found meh...


## Exploration using a word cloud

Generating a word cloud is one way by which you can check whether your data needs any further cleaning.

In [ ]:
from wordcloud import WordCloud

# join the words of the different tweets together into one string
long_string = ' '.join(unique_tweets)
new_long_string = ' '.join(set(long_string.split(" ")))

# create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')

# generate a word cloud
wordcloud.generate(new_long_string)

# visualize the word cloud
wordcloud.to_image()

# Topic Modelling

In [ ]:
!pip install -U gensim

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Do you want to modify this by adding more stop words?
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) 
    if word not in stop_words] for doc in texts]


data = tweets_df.text_processed.values.tolist()

data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

# create a dictionary
id2word = corpora.Dictionary(data_words)

# create a corpus
texts = data_words

# convert the corpus into a BoW representation
corpus = [id2word.doc2bow(text) for text in texts]


In [ ]:
from pprint import pprint

# set number of topics
num_topics = 5

# build an LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)

# print keywords in each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# visualise the topics
!pip install pyldavis

In [ ]:
import os
import pyLDAvis.gensim
import pickle 
import pyLDAvis

pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./'+str(num_topics))

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
with open(LDAvis_data_filepath, 'wb') as f:
  pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
  LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './'+ str(num_topics) +'.html')

LDAvis_prepared

# Sentiment Analysis

This implementation is based on the lexicon- and rule-based [VADER](https://github.com/cjhutto/vaderSentiment) sentiment analysis tool.

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

for tweet_text in unique_tweets:
    vs = analyzer.polarity_scores(tweet_text)
    print(tweet_text + '\t' + str(vs))

# Named Entity Recognition


This implementation is based on [spaCy's model](https://spacy.io/models/en#en_core_web_trf) using contextualised embeddings.

In [ ]:
!pip install spacy-transformers
!python -m spacy download en_core_web_trf
import spacy
import en_core_web_trf

nlp = spacy.load('en_core_web_trf')


In [ ]:
for tweet_text in unique_tweets:
  doc = nlp(tweet_text)
  print(tweet_text)
  for ne in doc.ents:
    print('\tNE found: ', ne.start_char, ne.end_char, ne.label_, tweet_text[ne.start_char:ne.end_char])


# Named Entity Linking

This implementation is based on [spaCy Entity Linker](https://github.com/egerber/spacy-entity-linker).

In [ ]:
!pip install spacy-entity-linker
!python -m spacyEntityLinker "download_knowledge_base"

In [ ]:
from spacyEntityLinker import EntityLinker
from spacy.language import Language

@Language.factory(
   "entityLinker"
)
def create_linker(nlp, name):
  return EntityLinker()

#add to pipeline
nlp.add_pipe('entityLinker')

In [ ]:
for tweet_text in unique_tweets:
  doc = nlp(tweet_text)
  print(tweet_text)
  all_linked_entities = doc._.linkedEntities
  all_linked_entities.pretty_print()
